In [1]:
import pandas as pd
import numpy as np
from utils import remove_unicode, remove_digits, remove_punctuation, remove_stopwords

In [2]:
df = pd.read_table("tagged.tsv")

In [3]:
df.head()

,title,content_urls,description,last_updated,n_comments,n_votes,prerequisites,section,speaker_info,speaker_links,target_audience,type,year,selected
0,"Don’t get scared, Get Started!",TBA,"Opensource world is full of excitement, knowle...","31 May, 2015",1,19,Nothing.....other than your passion for coding.,Others,Tapasweni PathakI have done bachelors in IT fr...,Tapasweni PathakShaifali Agrawal,Beginner,Talks,2015,False
1,Integrating human workflows with Python,NaN,"At iDataLabs, we need to use actual humans to ...","31 May, 2015",2,-5,None,Data Visualization and Analytics,Chinmay Kanchi is a data scientist and develop...,Pandas tutorial at Scipy India 2015Twitter,Beginner,Talks,2015,False
2,"Python 3 Metaprogramming, Macros, Madness & More!",Github repository for the code that accompanie...,"SummaryEver wanted to conquer the world, but j...","03 Sep, 2015",0,70,NaN,Core Python,"Suhas is a Data Scientist at Gramener, and was...",NaN,Intermediate,Talks,2015,True
3,Geo-Python! Python for spatial data,GitHub repoSlides etc. will be uploaded shortl...,Geoprocessing tasks can be time intensive sinc...,"21 May, 2015",0,4,Knowledge Prerequisites:Software Prerequisites...,Scientific Computing,Electronics engineer with 9 years of experienc...,NaN,Intermediate,Workshops,2015,False
4,Pandas for beginners (data science 101),https://github.com/cgkanchi/scipy_in_exploring...,"In this tutorial, I will demonstrate how to ge...","31 May, 2015",0,16,"Basic knowledge of PythonComputer with Python,...",Data Visualization and Analytics,Chinmay Kanchi is a data scientist and develop...,http://twitter.com/amphiesma,Beginner,Workshops,2015,False


In [4]:
text_cols = "title speaker_info section target_audience type prerequisites description".split()
pipe = [remove_unicode, remove_digits, remove_punctuation, remove_stopwords, lambda x: x.lower()]
for col in text_cols:
    s = df.pop(col)
    for cleaner in pipe:
        s = s.astype(str).apply(cleaner)
    df[col] = s

In [5]:
df.head()

,content_urls,last_updated,n_comments,n_votes,speaker_links,year,selected,title,speaker_info,section,target_audience,type,prerequisites,description
0,TBA,"31 May, 2015",1,19,Tapasweni PathakShaifali Agrawal,2015,False,dont scared get started,tapasweni pathaki bachelors it igdtuw i wrote ...,others,beginner,talks,nothingother passion coding,opensource world excitement knowledge encourag...
1,NaN,"31 May, 2015",2,-5,Pandas tutorial at Scipy India 2015Twitter,2015,False,integrating human workflows python,chinmay kanchi data scientist developer idatal...,data visualization analytics,beginner,talks,none,at idatalabs need use actual humans validate d...
2,Github repository for the code that accompanie...,"03 Sep, 2015",0,70,NaN,2015,True,python metaprogramming macros madness more,suhas data scientist gramener previously engin...,core python,intermediate,talks,nan,summaryever wanted conquer world just fell sho...
3,GitHub repoSlides etc. will be uploaded shortl...,"21 May, 2015",0,4,NaN,2015,False,geopython python spatial data,electronics engineer years experience working...,scientific computing,intermediate,workshops,knowledge prerequisitessoftware prerequisitesy...,geoprocessing tasks time intensive performed n...
4,https://github.com/cgkanchi/scipy_in_exploring...,"31 May, 2015",0,16,http://twitter.com/amphiesma,2015,False,pandas beginners data science,chinmay kanchi data scientist developer idatal...,data visualization analytics,beginner,workshops,basic knowledge pythoncomputer python numpy sc...,in tutorial i demonstrate load process publicl...


In [6]:
df['speaker_link_present'] = False
df['content_url_present'] = False

In [7]:
for col in df:
    if df[col].dtype is np.dtype('O'):
        df[col].fillna(value="", inplace=True)

In [8]:
URL_PATTERN = "(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
df.loc[df.speaker_links.str.contains(URL_PATTERN, case=False), "speaker_link_present"] = True
df.loc[df.content_urls.str.contains(URL_PATTERN, case=False), "content_url_present"] = True

/Users/jaidevd/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app
/Users/jaidevd/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  app.launch_new_instance()


In [9]:
df.content_url_present.sum() / float(df.shape[0])

0.1793103448275862

In [10]:
del df['content_urls']
del df['speaker_links']

In [11]:
df['last_updated'] = pd.to_datetime(df.last_updated, errors='raise')

In [12]:
df.head()

,last_updated,n_comments,n_votes,year,selected,title,speaker_info,section,target_audience,type,prerequisites,description,speaker_link_present,content_url_present
0,2015-05-31,1,19,2015,False,dont scared get started,tapasweni pathaki bachelors it igdtuw i wrote ...,others,beginner,talks,nothingother passion coding,opensource world excitement knowledge encourag...,False,False
1,2015-05-31,2,-5,2015,False,integrating human workflows python,chinmay kanchi data scientist developer idatal...,data visualization analytics,beginner,talks,none,at idatalabs need use actual humans validate d...,False,False
2,2015-09-03,0,70,2015,True,python metaprogramming macros madness more,suhas data scientist gramener previously engin...,core python,intermediate,talks,nan,summaryever wanted conquer world just fell sho...,False,False
3,2015-05-21,0,4,2015,False,geopython python spatial data,electronics engineer years experience working...,scientific computing,intermediate,workshops,knowledge prerequisitessoftware prerequisitesy...,geoprocessing tasks time intensive performed n...,False,False
4,2015-05-31,0,16,2015,False,pandas beginners data science,chinmay kanchi data scientist developer idatal...,data visualization analytics,beginner,workshops,basic knowledge pythoncomputer python numpy sc...,in tutorial i demonstrate load process publicl...,True,True


In [13]:
deadline_16 = pd.to_datetime("1 July 2016")
deadline_15 = pd.to_datetime("1 June 2015")

In [14]:
df['deadlinediff'] = 0
df.loc[df.year == 2015, "deadlinediff"] = deadline_15 - df[df.year == 2015]['last_updated']
df.loc[df.year == 2016, "deadlinediff"] = deadline_16 - df[df.year == 2016]['last_updated']

In [15]:
del df['last_updated']

In [16]:
df.head()

,n_comments,n_votes,year,selected,title,speaker_info,section,target_audience,type,prerequisites,description,speaker_link_present,content_url_present,deadlinediff
0,1,19,2015,False,dont scared get started,tapasweni pathaki bachelors it igdtuw i wrote ...,others,beginner,talks,nothingother passion coding,opensource world excitement knowledge encourag...,False,False,1 days
1,2,-5,2015,False,integrating human workflows python,chinmay kanchi data scientist developer idatal...,data visualization analytics,beginner,talks,none,at idatalabs need use actual humans validate d...,False,False,1 days
2,0,70,2015,True,python metaprogramming macros madness more,suhas data scientist gramener previously engin...,core python,intermediate,talks,nan,summaryever wanted conquer world just fell sho...,False,False,-94 days
3,0,4,2015,False,geopython python spatial data,electronics engineer years experience working...,scientific computing,intermediate,workshops,knowledge prerequisitessoftware prerequisitesy...,geoprocessing tasks time intensive performed n...,False,False,11 days
4,0,16,2015,False,pandas beginners data science,chinmay kanchi data scientist developer idatal...,data visualization analytics,beginner,workshops,basic knowledge pythoncomputer python numpy sc...,in tutorial i demonstrate load process publicl...,True,True,1 days


In [17]:
df['deadlinediff'] = df.deadlinediff.apply(lambda x: x.days)

AttributeError: 'NaTType' object has no attribute 'days'

In [ ]:
df.head()

In [ ]:
df.to_csv("preprocessed.tsv", index=False, sep="\t")